In [ ]:
!nvidia-smi

Tue Jun  7 13:50:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/phd-thesis/
!ls

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/phd-thesis
 Attack_Types.csv
'bagging&ensemble.py'
 cnn.ipynb
 createNetwork-211025.py
 createNetwork-testdata.ipynb
 createNetwork-testdata.py
 emsemble.ipynb
 ensemble_imb.ipynb
 Field_Names.csv
 intrusion-detection-classification-by-jinner.ipynb
 KDDTest+.csv
 KDDTest_x_20.csv
 KDDTest_x.csv
 KDDTest_x_num.csv
 KDDTest_y_20.csv
 KDDTest_y.csv
 KDDTrain+.csv
 KDDTrain_x_20.csv
 KDDTrain_x.csv
 KDDTrain_x_num.csv
 KDDTrain_y_20.csv
 KDDTrain_y.csv
 load_xgbModel.ipynb
 MySSL.py
 myXgb_imb.json
 myXgb.json
 myXgbModel.pkl
 prehandle-1009.py
 RUS-20.py
 smote-test.ipynb
 test_2.csv
 test_data.csv
 testdata_target.csv
 testdata_train.csv
 test-vc.ipynb
 test_xgboost-evaluaton.ipynb
 test_xgboost_nsl_kdd.ipynb
 train_2.csv
 train_2.rar
 Train_data.csv
 Untitled0.ipynb
 Untitled1.ipynb
 xgboost_test_imb.ipynb
 xgboost_test.ipynb


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 30 14:40:43 2021
Comparison using Majority Voting classifier and ensemble methods
using somote for oversampling
nsl-kdd dataset is 20%
@author: Administrator
"""
import pandas as pd
import numpy as np
import sys
#bagging and ensemble methods
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, zero_one_loss, \
    accuracy_score, classification_report, roc_auc_score
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced

#other ensemble algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from collections import Counter
import joblib

#traing dataset
#train_x = pd.read_csv('KDDTrain_x_20.csv',header=None)
#train_Y = pd.read_csv('KDDTrain_y_20.csv',header=None)
train_x = pd.read_csv('KDDTrain_x.csv',header=None)
train_Y = pd.read_csv('KDDTrain_y.csv',header=None)
train_x.drop(train_x.columns[[115,116,117]], axis=1, inplace=True)
train_Y[0]=list(train_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))

print("train_x", train_x.head())
print("train_y", train_Y.head())
###

#testing dataset
#test_x = pd.read_csv('KDDTest_x_20.csv',header=None)
#test_Y = pd.read_csv('KDDTest_y_20.csv',header=None)
test_x = pd.read_csv('KDDTest_x.csv',header=None)
test_Y = pd.read_csv('KDDTest_y.csv',header=None)
test_x.drop(test_x.columns[[115,116,117]], axis=1, inplace=True)
test_Y[0]=list(test_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))

print("test_x", test_x.head())
print("test_y", test_Y.head())

train_Y = train_Y.values.ravel()
test_Y  = test_Y.values.ravel()
print('train dataset shape %s' % Counter(train_Y))
print('test dataset shape %s' % Counter(test_Y))
#print("before SMOTE \n", pd.Series(train_Y).value_counts())

"""
#***apply SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
#***only smote
sm = SMOTE (sampling_strategy='auto', random_state=0)
train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
print('smote dataset shape %s' % Counter(train_Y_sm))

#***Tomek Links
from imblearn.under_sampling import TomekLinks
sampling_strategy = "all"
tl = TomekLinks(sampling_strategy=sampling_strategy)
# x_res, Y_res = tl.fit_resample(train_x_sm, train_Y_sm)
x_res, Y_res = tl.fit_resample(train_x_sm, train_Y_sm)
print('Tomek Links dataset shape %s' % Counter(Y_res))
"""

#***smote+tomek
# smt = SMOTETomek(random_state=42)
# train_x_smt, train_Y_smt = smt.fit_resample(train_x, train_Y)
# print('after smote+tomek shape %s' % Counter(train_Y_smt))

# print("after SMOTE",  pd.Series(train_Y_smt).value_counts())
#sys.exit(0)
##-------------------------

#Try Majority Voting Method
def votingMethod():
    # log_clf = LogisticRegression(solver='lbfgs', max_iter=100)
    log_clf = LogisticRegression(solver='liblinear',max_iter=3000)
    # rnd_clf = RandomForestClassifier()
    svm_clf = SVC(probability=True)
    # nb_clf = GaussianNB()
    dt_clf = DecisionTreeClassifier(random_state=17)   #try random_state=1
    kn_clf = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
    
    voting_clf=VotingClassifier(
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('rf',rnd_clf), ('svm',svm_clf)],
        # voting='soft'
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('kn',kn_clf), ('svm',svm_clf)],
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('kn', kn_clf), ('svm',svm_clf)],
        #estimators=[('lc',log_clf),('dt',dt_clf), ('kn', kn_clf), ('svm',svm_clf)],  #original  
        estimators=[('lc',log_clf), ('dt',dt_clf), ('kn', kn_clf), ('svm',svm_clf)],  #decision tree
        voting='soft'
    )
    
    #evaluation of voting classifier
    target_names = ['benign 0', 'dos 1', 'probe 2', 'r2l 3', 'u2r 4']
    #for clf in (log_clf, dt_clf, kn_clf, svm_clf, voting_clf):
    for clf in (log_clf, dt_clf, kn_clf, svm_clf, voting_clf): 
        #with and without smote
        #clf.fit(train_x, train_Y)   #without smote
        # clf.fit(train_x_sm, train_Y_sm)    #with SMOTE
        # clf.fit(x_res,Y_res)    #with SMOTE+tomek
        clf.fit(train_x,train_Y)    #with SMOTE+tomek
        pred_Y = clf.predict(test_x)
        #cross_val_score(clf, X_train, y2_train, cv=3, scoring="accuracy")
        #print(clf.__class__.__name__,accuracy_score(y2_test, y_pred))
        results = confusion_matrix(test_Y, pred_Y, labels= clf.classes_)
        error = zero_one_loss(test_Y, pred_Y) 
        print("validation:", clf.__class__.__name__,accuracy_score(test_Y, pred_Y))
        print("error:", error)
        print("result: \n" , results)
        
        joblib.dump(clf, "VotingClassifier-imb.pkl")

        yy = clf.predict_proba(test_x)
        rac = roc_auc_score(test_Y, yy, multi_class='ovr', average ='macro')  #Y_res
        print("ROC AUC Score:", rac)
        print(classification_report(test_Y, pred_Y, target_names=target_names))   #Y_res
        print(classification_report_imbalanced(test_Y, pred_Y, target_names=target_names))  #Y_res
        print(f"The geometric mean is {geometric_mean_score(test_Y, pred_Y):.3f}") #Y_res
        
        
        
votingMethod()
######
"""
#try other ensembleMethods
def ensembleMethods():
    #ensemble methods
    dt_clf = DecisionTreeClassifier(random_state=17) 
    rnd_clf = RandomForestClassifier()
    ada_clf = AdaBoostClassifier(base_estimator=dt_clf,
                             n_estimators= 10,   #n_estimators= 500,
                             learning_rate=0.1,
                             random_state=17)
    
    gbc_clf = GradientBoostingClassifier(max_depth=None, n_estimators=10, learning_rate=0.1,random_state=17)
    hgbc_clf = HistGradientBoostingClassifier(max_depth=None, loss="categorical_crossentropy", learning_rate=0.1,random_state=17)
    etc_clf = ExtraTreesClassifier(max_depth=None, n_estimators=10, random_state=0)
    
    for clf in (dt_clf, rnd_clf, ada_clf, gbc_clf, hgbc_clf, etc_clf):  #test random forest algorithm
        clf.fit(train_x, train_Y)  #train model
        pred_Y = clf.predict(test_x)     #validation
        results = confusion_matrix(test_Y, pred_Y, labels= clf.classes_)
        print("validation:", clf.__class__.__name__,accuracy_score(test_Y, pred_Y))
        print(results)
        
ensembleMethods()
######
"""

train_x         0         1         2    3         4         5         6        7    \
0 -0.113551 -0.009889 -0.039310    0 -0.091223 -0.006301 -0.091933 -0.02622   
1 -0.113551 -0.010032 -0.039310    0 -0.091223 -0.006301 -0.091933 -0.02622   
2 -0.113551 -0.010093 -0.039310    0 -0.091223 -0.006301 -0.091933 -0.02622   
3 -0.113551 -0.009996  0.052473    0 -0.091223 -0.006301 -0.091933 -0.02622   
4 -0.113551 -0.010010 -0.034582    0 -0.091223 -0.006301 -0.091933 -0.02622   

   8         9    ...  105  106  107  108  109  110  111  112  113  114  
0    0 -0.021873  ...    0    0    0    0    0    0    0    0    1    0  
1    0 -0.021873  ...    0    0    0    0    0    0    0    0    1    0  
2    0 -0.021873  ...    0    0    0    0    1    0    0    0    0    0  
3    1 -0.021873  ...    0    0    0    0    0    0    0    0    1    0  
4    1 -0.021873  ...    0    0    0    0    0    0    0    0    1    0  

[5 rows x 115 columns]
train_y         0
0  benign
1  benign
2     dos
3

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    benign 0       0.27      0.87      0.41      2152
       dos 1       0.92      0.58      0.71      4520
     probe 2       0.83      0.59      0.69      2404
       r2l 3       0.99      0.10      0.18      2574
       u2r 4       0.00      0.00      0.00       200

    accuracy                           0.52     11850
   macro avg       0.60      0.43      0.40     11850
weighted avg       0.78      0.52      0.52     11850



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                   pre       rec       spe        f1       geo       iba       sup

   benign 0       0.27      0.87      0.47      0.41      0.64      0.42      2152
      dos 1       0.92      0.58      0.97      0.71      0.75      0.54      4520
    probe 2       0.83      0.59      0.97      0.69      0.75      0.55      2404
      r2l 3       0.99      0.10      1.00      0.18      0.31      0.09      2574
      u2r 4       0.00      0.00      1.00      0.00      0.00      0.00       200

avg / total       0.78      0.52      0.88      0.52      0.62      0.41     11850

The geometric mean is 0.000
20% validation: VotingClassifier 0.5562025316455697
20% error: 0.44379746835443035
20% result: 
 [[1891   55  206    0    0]
 [1556 2911   53    0    0]
 [ 640  159 1604    1    0]
 [2391    2    9  171    1]
 [ 158    0   25    3   14]]
ROC AUC Score: 0.8242208620213676
              precision    recall  f1-score   support

    benign 0       0.28      0.88      0.43      2152
       

'\n#try other ensembleMethods\ndef ensembleMethods():\n    #ensemble methods\n    dt_clf = DecisionTreeClassifier(random_state=17) \n    rnd_clf = RandomForestClassifier()\n    ada_clf = AdaBoostClassifier(base_estimator=dt_clf,\n                             n_estimators= 10,   #n_estimators= 500,\n                             learning_rate=0.1,\n                             random_state=17)\n    \n    gbc_clf = GradientBoostingClassifier(max_depth=None, n_estimators=10, learning_rate=0.1,random_state=17)\n    hgbc_clf = HistGradientBoostingClassifier(max_depth=None, loss="categorical_crossentropy", learning_rate=0.1,random_state=17)\n    etc_clf = ExtraTreesClassifier(max_depth=None, n_estimators=10, random_state=0)\n    \n    for clf in (dt_clf, rnd_clf, ada_clf, gbc_clf, hgbc_clf, etc_clf):  #test random forest algorithm\n        clf.fit(train_x, train_Y)  #train model\n        pred_Y = clf.predict(test_x)     #validation\n        results = confusion_matrix(test_Y, pred_Y, labels= c